In [1]:
import os
import re
from datetime import datetime

start_timestamp = int(datetime.strptime("2017-10-01",
                                        "%Y-%m-%d").timestamp())

end_timestamp = int(datetime.strptime("2018-11-01", "%Y-%m-%d").replace(
                                          hour=23, minute=59, second=59
                                          ).timestamp())
print(start_timestamp)
print(end_timestamp)

1506816000
1541116799


In [ ]:
query=""" 
SELECT *, RANK() OVER (PARTITION BY main_Id ORDER BY visitStartTime ASC) AS seq2 
FROM `emirates-ga-cloud-staging.test_attribution.uk_data_jan_onwards_data` 
WHERE main_Id IN ( SELECT main_Id FROM `emirates-ga-cloud-staging.test_attribution.uk_data_jan_onwards_data` 
WHERE visitStartTime BETWEEN """ +str(start_timestamp)+""" AND """+str(end_timestamp)+""" )"""

print(query)
import google.datalab.bigquery as bq
chunks = bq.Query(query).execute().result().to_dataframe()
chunks.head()
chunks.to_csv("BQData.csv")

 
SELECT *, RANK() OVER (PARTITION BY main_Id ORDER BY visitStartTime ASC) AS seq2 
FROM `emirates-ga-cloud-staging.test_attribution.uk_data_jan_onwards_data` 
WHERE main_Id IN ( SELECT main_Id FROM `emirates-ga-cloud-staging.test_attribution.uk_data_jan_onwards_data` 
WHERE visitStartTime BETWEEN 1506816000 AND 1541116799 )


In [47]:
def combine_channels(channels):
    channels = [x.replace(" ", "") for x in list(channels)]
    return " > ".join(channels)

In [83]:
import pandas as pd
import numpy as np
chunks1 = pd.read_csv("raj.csv", chunksize=1000000,
					dtype = {
					'fullVisitorId': 'str',
					'main_Id': 'str',
					'visitStartTime': 'int32',
					'visitStartTime': 'int32',
					'sourceChannel': 'object',
					'Platform': 'object',
					'medium': 'object',
					'source': 'object',
					'campaign': 'object',
					'keyword': 'object',
					'adContent': 'object',
					'revenue': 'float32',
					'transaction_time': 'float32',
					'seq': 'int8'
					}
					)

In [84]:
for chunk in chunks1:
    chunk['channels'] = chunk['sourceChannel']
     #Exclude Operational Emails
    print("Reformatting data...", file=sys.stderr)
    chunk.loc[(chunk['channels'].str.contains('Operational')), 'channels'] = 'Direct'
    chunk['interaction_date'] = pd.to_datetime(chunk['visitStartTime'], unit = 's')
    chunk['interaction_date'] = chunk['interaction_date'].dt.date.astype(str)

NameError: name 'sys' is not defined

In [96]:
#print(chunk.loc[chunk["channels"] == "Direct"])
#chunk.columns
# Reclassify App Direct
#print("Reclassifying App Direct..", file=sys.stderr)
chunk['Platform'] = chunk['Platform'].astype(str)
chunk.loc[(chunk['Platform'].str.contains('iPhone')), 'channels'] = 'Direct'
chunk.loc[(chunk['Platform'].str.contains('Android')), 'channels'] = 'Direct'
chunk.loc[(chunk['Platform'].str.contains('iPad')), 'channels'] = 'Direct'   
chunk.loc[(chunk['Platform'].str.contains(' ')), 'channels'] = 'Direct'   
chunk.loc[(chunk['channels'].str.contains('App_Direct')), 'channels'] = 'Direct'   
temp_2 = (chunk.groupby('main_Id')['visitStartTime'].nunique()).reset_index()
temp_2 = temp_2[temp_2['visitStartTime'] < 2]
chunk.loc[(chunk['Platform'].str.contains('iPhone|Android|iPad')) & (chunk['main_Id'].isin(temp_2['main_Id'])), 'channels'] = 'App_Direct'

# Use source within Display Prospecting, to attribute revenue to the specific Display Publisher
#print("Reformat Display Publishers", file=sys.stderr)
chunk.loc[(chunk['channels'].str.contains('Prospecting')) & (chunk['source'].str.contains('kayak')) , 'channels'] = 'DisplayProspectingKayak'
chunk.loc[(chunk['channels'].str.contains('Prospecting')) & (chunk['source'].str.contains('tripadvisor')) , 'channels'] = 'DisplayProspectingTripAdvisor'
chunk.loc[(chunk['channels'].str.contains('Prospecting')) & (chunk['source'].str.contains('skyscanner')) , 'channels'] = 'DisplayProspectingSkyscanner'
chunk.loc[(chunk['channels'].str.contains('Prospecting')) & (chunk['source'].str.contains('travelzoo')) , 'channels'] = 'DisplayProspectingTravelzoo'

# Split into converting and non-converting journeys
#print("Splitting by converting/non-converting", file=sys.stderr)
visitor_list = (chunk.groupby(['main_Id'])['transactionId'].count()).reset_index()
visitor_list = visitor_list[visitor_list['transactionId'] > 0]
conv = chunk[chunk['main_Id'].isin(visitor_list['main_Id'])]
non_conv = chunk[~(chunk['main_Id'].isin(visitor_list['main_Id']))]

conv.loc[conv['revenue'].notnull() & conv['transactionId'].isnull(), 'revenue'] = np.nan
conv["transaction_time"] = conv.groupby(["main_Id"])['transaction_time'].transform(lambda x: x.bfill())
conv['transaction_date'] = pd.to_datetime(conv['transaction_time'], unit = 's')
conv['transaction_date'] = conv['transaction_date'].dt.date.astype(str)
conv["transactionId"] = conv.groupby(["main_Id", "transaction_date"])['transactionId'].transform(lambda x: x.bfill())
conv_temp = (conv.groupby(['transaction_time', "transactionId"])['revenue'].max()).reset_index()
conv = conv.merge(conv_temp, how = 'outer', on = ['transaction_time', "transactionId"])
conv['revenue'] = conv['revenue_y']
conv = conv[conv['transaction_time'].notnull()]
conv = conv.dropna(subset=['revenue'])
conv = conv.drop_duplicates(keep='last')

### Implement 90 day lookback window
#print("Implement lookback window for converting journeys", file=sys.stderr)
conv['Time_To_Transaction_Days'] = (conv['transaction_time'] - conv['visitStartTime'])/(60*60*24)
conv = conv[conv['Time_To_Transaction_Days'] <= 90]

# Rank over interactions for new and returning customers
conv['rank'] = conv.groupby('main_Id')['transaction_time'].rank(method='dense')

# Split by new and returning customers
#print("Splitting by new/returning", file=sys.stderr)
conv["state"] = "new"
conv.loc[conv['rank'] > 1, "state"] = "returning"

#print("Grouping new/returning...", file=sys.stderr)
conv = (conv.groupby(["transaction_date", "state", 'transactionId', 'revenue'])['channels'].apply(combine_channels)).reset_index()

conv['var_value'] = conv['revenue']/1000000.0

# Select only the rows we want
conv = conv[['transaction_date', 'state', 'channels', 'var_value']]

# SPLIT BY PPC
PPC_start = conv[conv['channels'].str.contains("PPCBrand")]
no_PPC = conv[~conv['channels'].str.contains("PPCBrand")]
len_PPC = int(len(PPC_start))
PPC_TV = int(0.02 * len_PPC)

# Picks SIZE numbers from 0 to a
if(len_PPC>0):
   	chosen_idx=np.random.choice(len_PPC,replace=False,size=PPC_TV)
   	PPC_start.iloc[chosen_idx, PPC_start.columns.get_loc('channels')] = 'Tac_Imp' + ' > ' + PPC_start['channels']

all_dfs = [PPC_start, no_PPC]

# Concatenate all te dataframes back together
conv = pd.concat(all_dfs).reset_index(drop=True)  

#Count Total value
conv_grouped = (conv.groupby(['transaction_date', 'state', 'channels'])['var_value'].agg(['sum', 'count'])).reset_index()
#print("conversions final data file", file=sys.stderr)
conv_grouped.to_csv('resultConv.csv')

/usr/local/envs/py3env/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [97]:
#print(chunk.loc[chunk["channels"] == "DisplayProspectingKayak"])
conv.columns

Index(['transaction_date', 'state', 'channels', 'var_value'], dtype='object')